In [1]:
import torch
from torch.utils.data.dataset import Dataset
from datasets import load_dataset 
from sklearn.model_selection import train_test_split 

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.model_selection import train_test_split

# Example data. 
# In reality, the strings are usually longer and there are 11 possible classes
model_name = "prajjwal1/bert-tiny"
dataset = load_dataset("jigsaw_toxicity_pred", data_dir="/workspaces/LLM-Experimentation-Capstone/00_source_data/jigsaw_toxicity")
dataset = dataset['train']
# Example data. 
# In reality, the strings are usually longer and there are 11 possible classes
texts = dataset['comment_text']

labels = [[float(x), float(y), float(z), float(a), float(b), float(c)] for x, y, z, a, b, c in zip(dataset['toxic'], dataset['severe_toxic'], dataset['obscene'], 
                                                   dataset['threat'], dataset['insult'], dataset['identity_hate'])]


train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    texts, labels, test_size=0.33, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

train_encodings = tokenizer(train_texts, padding="max_length", truncation=True, max_length=512)
eval_encodings = tokenizer(eval_texts, padding="max_length", truncation=True, max_length=512)

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [2]:
class TextClassifierDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextClassifierDataset(train_encodings, train_labels)
eval_dataset = TextClassifierDataset(eval_encodings, eval_labels)

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    problem_type="multi_label_classification",
    num_labels=6
)

training_arguments = TrainingArguments(
    output_dir=".",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8192,
    per_device_eval_batch_size=512,
    num_train_epochs=100,
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)
trainer.train()

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

: 